In [2]:
!pip install pulp
import numpy as np
import pandas as pd
from pulp import *
import math
from scipy.sparse import csr_matrix

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Load data from CSV
data = pd.read_excel('LocationForecastAll.xlsx')

Name = data['City'].to_numpy()
print(Name)

['Abaetetuba' 'Abel Figueiredo' 'Abreulândia' 'Açailândia' 'Acará'
 'Acorizal' 'Acrelândia' 'Afuá' 'Água Azul do Norte' 'Água Boa'
 'Aguiarnópolis' 'Alcântara' 'Alenquer' 'Aliança do Tocantins' 'Almas'
 'Almeirim' 'Alta Floresta' "Alta Floresta D'Oeste" 'Altamira'
 'Altamira do Maranhão' 'Alto Alegre' 'Alto Alegre do Maranhão'
 'Alto Alegre do Pindaré' 'Alto Alegre dos Parecis' 'Alto Araguaia'
 'Alto Boa Vista' 'Alto Garças' 'Alto Paraguai' 'Alto Paraíso'
 'Alto Parnaíba' 'Alto Taquari' 'Alvarães' 'Alvorada' "Alvorada D'Oeste"
 'Amajari' 'Amapá' 'Amapá do Maranhão' 'Amarante do Maranhão' 'Amaturá'
 'Anajás' 'Anajatuba' 'Anamã' 'Ananás' 'Ananindeua' 'Anapu' 'Angico'
 'Anori' 'Aparecida do Rio Negro' 'Apiacás' 'Apicum-Açu' 'Apuí'
 'Aragominas' 'Araguacema' 'Araguaçu' 'Araguaiana' 'Araguaína'
 'Araguainha' 'Araguanã' 'Araguatins' 'Arame' 'Arapoema' 'Araputanga'
 'Arari' 'Arenápolis' 'Aripuanã' 'Ariquemes' 'Arraias' 'Assis Brasil'
 'Atalaia do Norte' 'Augustinópolis' 'Augusto Corrêa' 'Auro

In [5]:
# Read all sheets from the Excel file
results = pd.read_excel('results_MinTimex.xlsx', sheet_name=None)

# Access the 'Chosen Facilities' sheet
transportation_results_sheet = results['Transportation Results']
chosen_facilities_sheet = results['Chosen Facilities']
network_sheet = results['Network']

chosen_facilities = chosen_facilities_sheet['Chosen Facilities'].to_numpy()
print(chosen_facilities)

['Afuá' 'Beruri' 'Boca do Acre' 'Brasiléia' 'Coari' 'Codajás' 'Humaitá'
 'Igarapé-Miri' 'Inhangapi' 'Itacoatiara' 'Lábrea' 'Limoeiro do Ajuru'
 'Luís Domingues' 'Magalhães Barata' 'Manicoré' 'Marapanim' 'Mocajuba'
 'Muaná' 'Nova Olinda do Maranhão' 'Novo Aripuanã' 'Óbidos'
 'Oeiras do Pará' 'Oriximiná' 'Ponta de Pedras' 'Porto Velho' 'Rio Branco'
 'São Domingos do Capim' 'São Miguel do Guamá'
 'São Sebastião da Boa Vista' 'Sena Madureira']


In [7]:
Quantity_Facilities = []
for i in range(len(chosen_facilities)):
    Quantity = 0
    for _, row in transportation_results_sheet.iterrows():
      if row['Destination'] == chosen_facilities[i]:
         Quantity += float(row['Quantity'])
    Quantity_Facilities.append(float(Quantity))
    

Factory = []
for i in range(len(chosen_facilities)):
    Factory.append(chosen_facilities[i])

print("Getting the Quantity and Factory name in the format based on it's position in the Name to call OD matrix")
print(Quantity_Facilities)
print(Factory)
    

Getting the Quantity and Factory name in the format based on it's position in the Name to call OD matrix
[7.520374338999997, 7.144356084, 11.01128688, 1.6466055, 31.578700600000005, 5.3199668, 3.0335333, 2.197403677, 3.3253632, 4.461399554, 6.8981218989999995, 5.815842103, 8.070636703, 0.021321559999999996, 7.068900666999999, 0.32067625299999997, 24.19349416, 13.2858901, 5.019729042000001, 7.5448316, 4.759907901, 12.548662262000002, 5.714592961999998, 6.36576478, 4.478878496000001, 6.7463904229999985, 10.0141968, 15.898667844, 34.197223, 3.2560776180000004]
['Afuá', 'Beruri', 'Boca do Acre', 'Brasiléia', 'Coari', 'Codajás', 'Humaitá', 'Igarapé-Miri', 'Inhangapi', 'Itacoatiara', 'Lábrea', 'Limoeiro do Ajuru', 'Luís Domingues', 'Magalhães Barata', 'Manicoré', 'Marapanim', 'Mocajuba', 'Muaná', 'Nova Olinda do Maranhão', 'Novo Aripuanã', 'Óbidos', 'Oeiras do Pará', 'Oriximiná', 'Ponta de Pedras', 'Porto Velho', 'Rio Branco', 'São Domingos do Capim', 'São Miguel do Guamá', 'São Sebastião da

In [9]:
Port_Name = ['Manaus','São Luís']
Port = []
for i in range(len(Port_Name)):
    loc = (np.where(Name == Port_Name[i]))
    Port.append(loc[0].item())
    
towns_to_skip = []
Town = []
for i in range(len(Factory)):
    skip = (np.where(Name == Factory[i]))
    Town.append(skip[0].item())
    
TotalTown = list(range(767))
rows_to_skip = [item for item in TotalTown if item not in Town]

In [11]:
# Load distance matrices (/1000 to be km)
dist_matrix_River = (pd.read_excel('RiverDistance.xlsx', usecols = Port, skiprows = rows_to_skip, header=None).values / 1000)
dist_matrix_Road = (pd.read_excel('RoadDistance.xlsx', usecols = Port,skiprows = rows_to_skip, header=None).values / 1000)
dist_matrix_Rail = (pd.read_excel('RailDistance.xlsx', usecols = Port,skiprows = rows_to_skip, header=None).values / 1000)
print("River OD matrix")
print(dist_matrix_River)

River OD matrix
[[1541.08040013  824.02432296]
 [ 287.0398486  2356.27936208]
 [1925.27737025 3994.51688374]
 [2313.24219875 4382.48171224]
 [ 486.55413276 2555.79364625]
 [ 344.09018686 2413.32970034]
 [1015.8813669  2803.85473899]
 [1677.81222681    0.        ]
 [1715.70357104    0.        ]
 [ 215.8035367  1921.2801863 ]
 [1356.99092473 3426.23043821]
 [1614.93429932  897.87822214]
 [1932.50401807    0.        ]
 [1741.38531253    0.        ]
 [ 650.25489486 2438.22826696]
 [1731.30628561    0.        ]
 [1761.77575042 1139.12302784]
 [1625.21700666    0.        ]
 [2002.56507652    0.        ]
 [ 493.51276773 2281.48613983]
 [ 640.60593619 1497.56104798]
 [1711.85610017 1089.20337759]
 [ 623.53211926 1552.56175977]
 [1623.53433503    0.        ]
 [1272.91672001 3060.89009211]
 [5105.46011071 8962.67299629]
 [1740.27835807    0.        ]
 [1767.53770511    0.        ]
 [1586.6610971   869.60501993]
 [2099.17090386 4168.41041734]]


In [13]:
# Define transportation modes and their properties
Mode = ['Ferry with Freezer', 'Mid Size ferry', 'Large ferry', 'River Barges', 'Refrigerator truck','Normal Truck','Standard Cargo Train','Refrigerated Cargo Train']
# Mode Capacity in tonnes
mode_capacity = [125, 150, 900,3500, 18, 22.5,7000,7000]
# Mode fix cost ($/vehicle)
mode_fcost = [600000, 2000000,4000000,1250000, 150000, 100000,2000000,5000000]
# Mode variable cost (Per tonne per km)
mode_vcost = [ 0.125, 0.0435, 0.0435, 0.03, 3, 1.75, 0.02, 0.035]
# Mode speed (km/h)
mode_speed = [20, 21, 21, 12, 80, 80, 60, 60]
  

In [14]:
# Define decision variable
S = ['S' + str(i + 1) for i in range(len(Port))]  # Port
D = ['D' + str(i + 1) for i in range(len(Factory))]  # Factory
M = ['M' + str(i + 1) for i in range(len(Mode))]  # Mode
   

In [15]:
# Define parameters
# Quantity for each facility
QF = {customer: Quantity_Facilities[i] for i, customer in enumerate(D)}

# Customer-Facility distance based on mode
d = {customer: {facility: {mode: (dist_matrix_River[i, j] if k < 4 else dist_matrix_Road[i, j] if k >= 4 and k < 6 else dist_matrix_Rail[i,j] )
            for k, mode in enumerate(M)
        }
        for j, facility in enumerate(S)
    }
    for i, customer in enumerate(D)
}

# Mode capacity
m = {mode: mode_capacity[k] for k, mode in enumerate(M)}

# Mode total Cost (Sum of fix cost(Customer demand * mode fix cost/mode capacity) and variable cost(distance for ij route using k mode * customer demand * mode variable cost))
mc = {customer: {facility: {mode: (
                math.ceil(QF[customer] / m[mode]) * mode_fcost[k] * (d[customer][facility][mode] if int(d[customer][facility][mode]) == 0 else 1) +
                (d[customer][facility][mode]) * mode_vcost[k] * QF[customer] * math.ceil(QF[customer] / m[mode]))
                for k, mode in enumerate(M)
        }
        for j, facility in enumerate(S)
    }
    for i, customer in enumerate(D)
}

# Mode Speed
s = {mode: mode_speed[k] for k, mode in enumerate(M)}


In [19]:
# Decision variables
x = LpVariable.dicts('x', (D, S, M), lowBound=0, cat=LpContinuous)  # Quantity on route D to S using M
z = LpVariable.dicts('z', (D, S, M), lowBound=0, upBound=1, cat=LpInteger)  # Route.mode used open or not

In [20]:
# Define Objective function options
def objective_function(option):
    if option == 1:  # Minimise cost for initial investment and 1 day operation
        return lpSum([mc[i][j][k] * x[i][j][k]
                    for i in D
                    for j in S
                    for k in M
                    ])
    elif option == 2:  # Minimise time
        return lpSum([(d[i][j][k] / s[k]) * x[i][j][k]
                    for i in D
                    for j in S
                    for k in M
                    ])
    elif option == 3:  # Minimise cost and time
        return lpSum([(mc[i][j][k]/10000) * x[i][j][k] #Default is 10000
                    for i in D
                    for j in S
                    for k in M
                    ]) + lpSum([(d[i][j][k] / s[k]) * x[i][j][k]
                    for i in D
                    for j in S
                    for k in M
                    ])
                                
# Set the objective function (option 1, 2, or 3)
objective_option = 1
prob = LpProblem('prob', LpMinimize)
prob += objective_function(objective_option)


In [22]:
# Define constraints
# Each customer's product must be delivered
for i in D:
    prob += lpSum([x[i][j][k] for j in S for k in M]) == QF[i]
    
# Prevent trip with zero distance
for i in D:
    for j in S:
        for k in M:
            if d[i][j][k] == 0 and i != j:
                prob += x[i][j][k] == 0
                
    # z and x relationship
for i in D:
    for j in S:
        for k in M:
            prob += x[i][j][k] <= QF[i] * z[i][j][k]
          

# Travel time constraint: time must be less than 24 hours
for i in D:
    for j in S:
        for k in M:
            if k not in ['M1', 'M5','M8']:
                prob += (d[i][j][k] / s[k]) * z[i][j][k] <= 24


In [23]:
# Solve the problem
status = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/repl/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a2cfeecc105548c0b24fb373c9387646-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/a2cfeecc105548c0b24fb373c9387646-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1937 COLUMNS
At line 6838 RHS
At line 8771 BOUNDS
At line 9252 ENDATA
Problem MODEL has 1932 rows, 960 columns and 3616 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5.83752e+06 - 0.00 seconds
Cgl0004I processed model has 183 rows, 312 columns (157 integer (157 of which binary)) and 467 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 1.33227e-15
Cbc0038I Solution found of 5.90141e+06
Cbc0038I Relaxing continuous gives 5.90141e+06
Cbc0038I Before mini branch and bound, 157 integers at bound fixed and 129 contin

In [24]:
# Output the results
totalcost = 0
totaltime = 0
chosen_facilities = []
for i in D:
    for j in S:
        for k in M:
            if x[i][j][k].varValue > 0:
                # Determine which distance matrix is used
                mode_index = int(k[1:]) - 1
                if mode_index < 4:
                    distance_matrix = "River"
                    distance = dist_matrix_River[int(i[1:]) - 1, int(j[1:]) - 1]
                elif mode_index >= 4 and mode_index < 6:
                    distance_matrix = "Road"
                    distance = dist_matrix_Road[int(i[1:]) - 1, int(j[1:]) - 1]
                else:
                    distance_matrix = "Rail"
                    distance = dist_matrix_Rail[int(i[1:]) - 1, int(j[1:]) - 1]
                # Calculate travel time based on the chosen mode
                travel_time = distance / s[k]
                num_vehicles = math.ceil(x[i][j][k].varValue / m[k])
                route_cost = mc[i][j][k]
                totalcost += route_cost
                totaltime += travel_time * num_vehicles  # Sum travel time for all vehicles
                print(f"Customer {i} ({Factory[int(i[1:]) - 1]}) to Facility {j} ({Port_Name[int(j[1:]) - 1]}) using {Mode[mode_index]}")
                print(f"  Quantity: {x[i][j][k].varValue}")
                print(f"  Number of vehicles: {num_vehicles}")
                print(f"  Travel time: {travel_time} hours per vehicle")
                print(f"  Total travel time for all vehicles: {travel_time * num_vehicles} hours")
                print(f"  Cost for this route: {route_cost}")
                print(f"  Distance matrix used: {distance_matrix}")
                print(f"  Distance: {distance} km")


Customer D1 (Afuá) to Facility S2 (São Luís) using Ferry with Freezer
  Quantity: 7.5203743
  Number of vehicles: 1
  Travel time: 41.201216147845 hours per vehicle
  Total travel time for all vehicles: 41.201216147845 hours
  Cost for this route: 600774.6214216346
  Distance matrix used: River
  Distance: 824.0243229569 km
Customer D2 (Beruri) to Facility S1 (Manaus) using Normal Truck
  Quantity: 7.1443561
  Number of vehicles: 1
  Travel time: 3.2033561325 hours per vehicle
  Total travel time for all vehicles: 3.2033561325 hours
  Cost for this route: 103204.02836242231
  Distance matrix used: Road
  Distance: 256.2684906 km
Customer D3 (Boca do Acre) to Facility S1 (Manaus) using Normal Truck
  Quantity: 11.011287
  Number of vehicles: 1
  Travel time: 14.907713024999998 hours per vehicle
  Total travel time for all vehicles: 14.907713024999998 hours
  Cost for this route: 122981.43467801827
  Distance matrix used: Road
  Distance: 1192.6170419999999 km
Customer D4 (Brasiléia) to 

In [25]:
print(f"Network total transportation cost: {totalcost}")
print(f"Network total time: {totaltime} hours")
print(f'STATUS\n{LpStatus[status]}\n')


Network total transportation cost: 5901412.747641828
Network total time: 606.7802843583252 hours
STATUS
Optimal



In [ ]:
# Initialize lists to store data
results_data = []
facility_data = []

# Iterate over the decision variables to collect results
for i in D:
    for j in S:
        for k in M:
            if x[i][j][k].varValue > 0:
                mode_index = int(k[1:]) - 1
                if mode_index < 4:
                    distance_matrix = "River"
                    distance = dist_matrix_River[int(i[1:]) - 1, int(j[1:]) - 1]
                elif mode_index >= 4 and mode_index < 6:
                    distance_matrix = "Road"
                    distance = dist_matrix_Road[int(i[1:]) - 1, int(j[1:]) - 1]
                else:
                    distance_matrix = "Rail"
                    distance = dist_matrix_Rail[int(i[1:]) - 1, int(j[1:]) - 1]

                travel_time = distance / s[k]
                num_vehicles = math.ceil(x[i][j][k].varValue / m[k])
                route_cost = mc[i][j][k]   
                results_data.append([
                    Factory[int(i[1:]) - 1],
                    Port_Name[int(j[1:]) - 1],
                    Mode[mode_index],
                    x[i][j][k].varValue,
                    num_vehicles,
                    travel_time,
                    travel_time * num_vehicles,
                    route_cost,
                    distance_matrix,
                    distance
                ])

# Convert lists to DataFrame
results_df = pd.DataFrame(results_data, columns=[
    'Origin', 'Destination', 'Mode', 'Quantity', 'Number of Vehicles', 
    'Travel Time (hrs/vehicle)', 'Total Travel Time (hrs)', 
    'Cost for this Route', 'Distance Matrix Used', 'Distance (km)' 
])

AllCost = totalcost + network_sheet['Echelon1 Cost']
AllTime = totaltime + network_sheet['Echelon1 Time']
network_data = [[totalcost,totaltime,
    float(AllCost[0]),
    float(AllTime[0])]
]
network_df = pd.DataFrame(network_data, columns=['Echelon2 Cost', 'Echelon2 Time','Network Cost','Network Time'])

# Save to Excel
with pd.ExcelWriter('results_MinTime_MinCost.xlsx') as writer:
    results_df.to_excel(writer, sheet_name='Transportation Results', index=False)
    network_df.to_excel(writer, sheet_name='Network', index=False)


